In [2]:
!pip install --upgrade git+https://github.com/tweepy/tweepy@master
!pip install geopandas
!pip install geojson
!pip install searchtweets-v2==1.0.7

  Cloning https://github.com/tweepy/tweepy (to revision master) to /tmp/pip-req-build-2_4syvwx
  Running command git clone -q https://github.com/tweepy/tweepy /tmp/pip-req-build-2_4syvwx
  Created wheel for tweepy: filename=tweepy-3.10.0-py2.py3-none-any.whl size=26745 sha256=44d8622a94bf8710424edbfcd74155a683a309d5d4f645333affdb5e2fb5670f
  Stored in directory: /tmp/pip-ephem-wheel-cache-_8lrtqvr/wheels/12/5f/00/8cd18bd44822e1d48d9374f9db7e7f23e83cbde64e4a358680
Successfully built tweepy
  Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0


     |████████████████████████████████| 1.0MB 4.3MB/s 
     |████████████████████████████████| 6.5MB 36.4MB/s 
     |████████████████████████████████| 14.8MB 266kB/s 


In [3]:
import tweepy as tw
import json
import csv
import pandas as pd
import numpy as np
import requests, geopandas, geojson
import os
from datetime import datetime
import time
import searchtweets
from searchtweets import ResultStream, load_credentials, collect_results,gen_request_parameters
    

### Set up API credentials 

In [7]:
# twitter api authentication 
consumer_key = 'YOUR_CONSUMER_KEY'
consumer_secret = 'YOUR_CONSUMER_SECRET_KEY' 
access_token = 'YOUR ACCESS TOKEN'
access_token_secret = 'YOUR ACCESS TOKEN SECRET'


auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)


### Get location coordinates

In [5]:
# get districts (name, coordinates/geometry, and bounding box)

# retrieve region division to overlay on basic UK map (json file)

# this will be used for all districts of England (326)
url = 'https://martinjc.github.io/UK-GeoJSON/json/eng/topo_lad.json'

geo_data = requests.get(url).json()
geo_data = json.dumps(geo_data)

# the source chosen provides a weirdly formatted json file, so I'm converting 
# to a dataframe to properly format it, then back into a json file 

geo_data_df = geopandas.read_file(geo_data)

# this will be used for all districts of England (326)
geo_data_df = geo_data_df.drop(columns=['id','LAD13CD','LAD13CDO','LAD13NMW'])
geo_data_df = geo_data_df.rename(columns={"LAD13NM":"district"})

geo_objects = geo_data_df 

# convert polygons and multiploygons into bounding boxes (to use in Twitter search)
geo_data_df['bounding box'] = [x.bounds for x in geo_data_df['geometry']]
geo_data_df['tweets']=[None]*len(geo_data_df)

geo_data_df

,district,geometry,bounding box,tweets
0,Hartlepool,"MULTIPOLYGON (((-1.24244 54.72219, -1.24162 54...","(-1.3839929357874716, 54.62168846925439, -1.15...",None
1,Middlesbrough,"MULTIPOLYGON (((-1.19744 54.58184, -1.19416 54...","(-1.2849872220884917, 54.50096605164154, -1.13...",None
2,Redcar and Cleveland,"MULTIPOLYGON (((-1.20071 54.57768, -1.19989 54...","(-1.201527860127369, 54.487882833969216, -0.78...",None
3,Stockton-on-Tees,"MULTIPOLYGON (((-1.28171 54.56460, -1.27762 54...","(-1.452724175049573, 54.46409516547407, -1.158...",None
4,Darlington,"POLYGON ((-1.43800 54.59493, -1.43554 54.59552...","(-1.7096480932436195, 54.451011947801746, -1.4...",None
...,...,...,...,...
321,Sutton,"POLYGON ((-0.13456 51.39073, -0.13292 51.39013...","(-0.24583634276862387, 51.321744157265876, -0....",None
322,Tower Hamlets,"POLYGON ((-0.01673 51.54356, -0.01591 51.54237...","(-0.07973584788521215, 51.48587906988235, 0.00...",None
323,Waltham Forest,"POLYGON ((0.02172 51.62861, 0.02091 51.62920, ...","(-0.06255303806968637, 51.55010577481924, 0.02...",None
324,Wandsworth,"POLYGON ((-0.14028 51.41927, -0.14438 51.41749...","(-0.2589280073899767, 51.417489522958824, -0.1...",None


### Example of a tweet objet


In [11]:
# Example tweet objects - this is how a tweet looks like
coords = geo_data_df['bounding box'][9]
query= f'(immigration OR immigrants) bounding_box:[{coords[0]} {coords[1]} {coords[2]} {coords[3]}]'
      
example_tweet = api.search_full_archive(environment_name='fullarchive', 
                                        query = query, 
                                        fromDate = '201601010000', 
                                        toDate = '201701010000',
                                        maxResults=10,
                                        parser=tw.parsers.JSONParser())

#convert the first tweet to string
json_str = json.dumps(example_tweet['results'][0])

#deserialise string into python object
parsed = json.loads(json_str)

print(json.dumps(parsed, indent=4, sort_keys=True))


{
    "contributors": null,
    "coordinates": {
        "coordinates": [
            -0.27301979,
            53.75959015
        ],
        "type": "Point"
    },
    "created_at": "Fri Jan 22 11:58:35 +0000 2016",
    "entities": {
        "hashtags": [],
        "symbols": [],
        "urls": [],
        "user_mentions": []
    },
    "favorite_count": 0,
    "favorited": false,
    "filter_level": "low",
    "geo": {
        "coordinates": [
            53.75959015,
            -0.27301979
        ],
        "type": "Point"
    },
    "id": 690503826143629312,
    "id_str": "690503826143629312",
    "in_reply_to_screen_name": null,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": null,
    "is_quote_status": false,
    "lang": "en",
    "matching_rules": [
        {
            "tag": null
        }
    ],
    "place": {
        "attributes": {},
        "bounding_box": {
            "coordina

### Retrieving tweets

#### Tweets from 2016

In [ ]:
# yaml file with credentials 
import yaml

dict_file = {'search_tweets_v2':
             {'consumer_key':'vA4hP8SySsgeF1dY1Z5Cs4HF9',
              'consumer_secret':'s1qAAKRoAhRfImrS3CtKMvRdzIMrK0FZ0d9rPhnYLdt43i3QPO',
              'endpoint':'https://api.twitter.com/2/tweets/search/all',
              'bearer_token':'AAAAAAAAAAAAAAAAAAAAAIB6MgEAAAAAwlTnIEjZmJXYgacFb0KbMyfPUxY%3Dea9tzx094pTnK9PhUij7UkboDCatShNY6ExpAl1WlsJ8JZOi2h'}}
              
with open(r"./twitter_keys.yaml", 'w') as file:
    documents = yaml.dump(dict_file, file)


In [ ]:
# function that pull tweets according to parameters 
def pull_searchtweets(SEARCH_QUERY, RESULTS_PER_CALL, FROM_DATE, TO_DATE, MAX_RESULTS, DISTRICT_NAME):

    search_args = load_credentials(filename="./twitter_keys.yaml",
                    yaml_key="search_tweets_v2",
                    env_overwrite=False)

    rule = gen_request_parameters(SEARCH_QUERY,
                            results_per_call=RESULTS_PER_CALL,
                            start_time=FROM_DATE,
                            end_time=TO_DATE
                            )
    
    rs = ResultStream(request_parameters=rule,
                    max_results=MAX_RESULTS,
                    **search_args)
    n = 0
    tweet_list = []
    try:
        for tweet in rs.stream():
            n += 1
            tweet_list.append(tweet)
            if n >= MAX_RESULTS:
                break
    except: 
        print(f'An error happened while retrieving {DISTRICT_NAME} data')
    
    return tweet_list


In [ ]:
#create file that will keep data as it is retrieved
with open('raw_tweets.json', 'a') as f:
    json.dump({}, f)

for i in range(len(geo_data_df)):
    coords = geo_data_df['bounding box'][i]
    DISTRICT_NAME = geo_data_df['district'][i]
    SEARCH_QUERY = f'(immigration OR immigrants OR migration OR migrants \
                    OR "border control" OR emigration OR naturalize OR \
                    naturalization OR citizenship OR non-citizens OR borders) \
                    lang:en bounding_box:[{coords[0]} {coords[1]} {coords[2]} {coords[3]}]'
    RESULTS_PER_CALL = 500
    FROM_DATE = '2016-01-01' 
    TO_DATE = '2017-01-01'  

    MAX_RESULTS = 20000  # Total number of Tweets you want to collect (per district)

    tweet_list = pull_searchtweets(SEARCH_QUERY, RESULTS_PER_CALL, FROM_DATE, TO_DATE, MAX_RESULTS, DISTRICT_NAME)
    
    new_data = {DISTRICT_NAME: tweet_list}
    
    with open('raw_tweets.json', 'r') as f:
        updated_data = json.loads(f.read())

    updated_data.update(new_data)

    with open('raw_tweets.json', 'w') as f:
        json.dump(updated_data, f)
        f.write('\n')
        f.close()
    
    
print('done')


#### Tweets from 2020

In [ ]:
#create file that will keep data as it is retrieved
with open('raw_tweets_2020.json', 'a') as f:
    json.dump({}, f)

for i in range(len(geo_data_df)):
    coords = geo_data_df['bounding box'][i]
    DISTRICT_NAME = geo_data_df['district'][i]
    SEARCH_QUERY = f'(immigration OR immigrants OR migration OR migrants \
                    OR "border control" OR emigration OR naturalize OR \
                    naturalization OR citizenship OR non-citizens OR borders) \
                    lang:en bounding_box:[{coords[0]} {coords[1]} {coords[2]} {coords[3]}]'
    RESULTS_PER_CALL = 500
    FROM_DATE = '2020-03-01' 
    TO_DATE = '2021-03-01'  

    MAX_RESULTS = 20000  # Total number of Tweets you want to collect (per district)

    tweet_list = pull_searchtweets(SEARCH_QUERY, RESULTS_PER_CALL, FROM_DATE, TO_DATE, MAX_RESULTS, DISTRICT_NAME)
    
    new_data = {DISTRICT_NAME: tweet_list}
    
    with open('raw_tweets_2020.json', 'r') as f:
        updated_data = json.loads(f.read())

    updated_data.update(new_data)

    with open('raw_tweets_2020.json', 'w') as f:
        json.dump(updated_data, f)
        f.write('\n')
        f.close()
    
    
print('done')


 HTTP Error code: 429: {"title":"Too Many Requests","type":"about:blank","status":429,"detail":"Too Many Requests"} | Too Many Requests
 Request payload: {'query': '(immigration OR immigrants OR migration OR migrants OR "border control" OR emigration OR naturalize OR naturalization OR citizenship OR non-citizens OR borders) lang:en bounding_box:[-1.3839929357874716 54.62168846925439 -1.1573434920303036 54.726948902345384]', 'max_results': 500, 'start_time': '2020-03-01T00:00:00Z', 'end_time': '2021-03-01T00:00:00Z', 'next_token': 'b26v89c19zqg8o3fosbteinkyxrhqtr8vrm7epsnuewe5'}
Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP Error code: 429: {"title":"Too Many Requests","type":"about:blank","status":429,"detail":"Too Many Requests"} | Too Many Requests
 Request payload: {'query': '(immigration OR immigrants OR migration OR migrants OR "border control" OR emigration OR naturalize OR naturalization OR citizenship OR non-citizens OR borders) lang:en bounding_box:[-1.38399

An error happened while retrieving East Riding of Yorkshire data


 HTTP Error code: 400: {"errors":[{"parameters":{"query":["(immigration OR immigrants OR migration OR migrants OR \"border control\" OR emigration OR naturalize OR naturalization OR citizenship OR non-citizens OR borders) lang:en bounding_box:[-0.9503315452051648 53.45549802128007 -0.20410666178806203 53.714783607877116]"]},"message":"There were errors processing your request: Width of bounding_box must be less than 25 miles (at position 170)"}],"title":"Invalid Request","detail":"One or more parameters to your request was invalid.","type":"https://api.twitter.com/2/problems/invalid-request"} | Bad Request
 Request payload: {'query': '(immigration OR immigrants OR migration OR migrants OR "border control" OR emigration OR naturalize OR naturalization OR citizenship OR non-citizens OR borders) lang:en bounding_box:[-0.9503315452051648 53.45549802128007 -0.20410666178806203 53.714783607877116]', 'max_results': 500, 'start_time': '2020-03-01T00:00:00Z', 'end_time': '2021-03-01T00:00:00Z'}

An error happened while retrieving North Lincolnshire data


 HTTP Error code: 429: {"title":"Too Many Requests","type":"about:blank","status":429,"detail":"Too Many Requests"} | Too Many Requests
 Request payload: {'query': '(immigration OR immigrants OR migration OR migrants OR "border control" OR emigration OR naturalize OR naturalization OR citizenship OR non-citizens OR borders) lang:en bounding_box:[-1.5566392629815597 52.86080630890153 -1.3831747067486369 52.96785081712967]', 'max_results': 500, 'start_time': '2020-03-01T00:00:00Z', 'end_time': '2021-03-01T00:00:00Z'}
Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP Error code: 429: {"title":"Too Many Requests","type":"about:blank","status":429,"detail":"Too Many Requests"} | Too Many Requests
 Request payload: {'query': '(immigration OR immigrants OR migration OR migrants OR "border control" OR emigration OR naturalize OR naturalization OR citizenship OR non-citizens OR borders) lang:en bounding_box:[-1.5566392629815597 52.86080630890153 -1.3831747067486369 52.96785081712

...

An error happened while retrieving Malvern Hills data


 HTTP Error code: 429: {"title":"Too Many Requests","type":"about:blank","status":429,"detail":"Too Many Requests"} | Too Many Requests
 Request payload: {'query': '(immigration OR immigrants OR migration OR migrants OR "border control" OR emigration OR naturalize OR naturalization OR citizenship OR non-citizens OR borders) lang:en bounding_box:[-2.263589152534604 52.16144885514437 -2.157219377486114 52.23162247720504]', 'max_results': 500, 'start_time': '2020-03-01T00:00:00Z', 'end_time': '2021-03-01T00:00:00Z'}
Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP Error code: 429: {"title":"Too Many Requests","type":"about:blank","status":429,"detail":"Too Many Requests"} | Too Many Requests
 Request payload: {'query': '(immigration OR immigrants OR migration OR migrants OR "border control" OR emigration OR naturalize OR naturalization OR citizenship OR non-citizens OR borders) lang:en bounding_box:[-2.275862588117122 51.99969270937741 -1.757105377496023 52.361265270503566

An error happened while retrieving Bexley data


 HTTP Error code: 429: Rate limit exceeded
 | Too Many Requests
 Request payload: {'query': '(immigration OR immigrants OR migration OR migrants OR "border control" OR emigration OR naturalize OR naturalization OR citizenship OR non-citizens OR borders) lang:en bounding_box:[-0.3358415370404231 51.52750748974885 -0.191833226205544 51.60065457037141]', 'max_results': 500, 'start_time': '2020-03-01T00:00:00Z', 'end_time': '2021-03-01T00:00:00Z'}
Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP Error code: 429: Rate limit exceeded
 | Too Many Requests
 Request payload: {'query': '(immigration OR immigrants OR migration OR migrants OR "border control" OR emigration OR naturalize OR naturalization OR citizenship OR non-citizens OR borders) lang:en bounding_box:[-0.3358415370404231 51.52750748974885 -0.191833226205544 51.60065457037141]', 'max_results': 500, 'start_time': '2020-03-01T00:00:00Z', 'end_time': '2021-03-01T00:00:00Z'}
Rate limit hit... Will retry...
Will retry in

done
